# MongoDB Field Level Encryption Demo

In [ ]:
import pymongo
from pymongo import MongoClient
from pymongo.encryption import ClientEncryption
from bson import Binary
import pymongocrypt
import base64
from bson.codec_options import DEFAULT_CODEC_OPTIONS

### Initialise Key Vault:

Example Shell Command: head -c 96 /dev/urandom | base64 > key.txt

In [ ]:
cmk = base64.b64decode("IkhJDuh1kQXt7cNlrBkKEMpV/jbpUERloqPGCB0WaNhFC7Ig4z1X2dDwv6XByMlrRyadd3MEC+BdcY8D9cadCgvHe3MVa8o1m00+xcE4KDFJaID/FS1rwZKRPK3KfezP")

kms_providers = { "local" : { "key" : cmk } }

key_vault_namespace = "encryption.__dataKeys"

In [ ]:
keyVaultClient = pymongo.MongoClient("127.0.0.1")

In [ ]:
keyVault = ClientEncryption(kms_providers, key_vault_namespace, keyVaultClient, DEFAULT_CODEC_OPTIONS)

### Create Data Key

Create two data encryption keys:

In [ ]:
keyVault.create_data_key("local", master_key=cmk, key_alt_names=["key1"])

In [ ]:
keyVault.create_data_key("local", master_key=cmk, key_alt_names=["key2"])

Retrieve the key _id to be referenced in the schema_map:

In [ ]:
keyDb = keyVaultClient.encryption
key1 = keyDb['__dataKeys'].find_one({'keyAltNames':'key1'})['_id']
key2 = keyDb['__dataKeys'].find_one({'keyAltNames':'key2'})['_id']

Define the encryption schema:

In [ ]:
schema_map = {
    "hr.employees":
    {
        'bsonType': 'object',
        'encryptMetadata': {
            'keyId': [key1]
            },
        'properties': {
            'ssn': {
                'encrypt': {
                    'keyId': [key2],
                    'algorithm': 'AEAD_AES_256_CBC_HMAC_SHA_512-Random'
                    }
                },
            'ssn-last': {
                'encrypt': {
                    'algorithm': 'AEAD_AES_256_CBC_HMAC_SHA_512-Deterministic',
                    'bsonType': 'string'
                    }
                },
            'position': {
                'bsonType': 'object',
                'encryptMetadata': {
                    'algorithm': 'AEAD_AES_256_CBC_HMAC_SHA_512-Deterministic'
                    },
                'properties': {
                    'compensation': {
                        'encrypt': {
                            'bsonType': 'int'
                            }
                        }
                }}
            }
        }
    }

Configure the field level encryption client:

In [ ]:
fleOptions = pymongo.encryption_options.AutoEncryptionOpts(kms_providers, key_vault_namespace, key_vault_client=None, schema_map=schema_map, bypass_auto_encryption=False, mongocryptd_uri='mongodb://localhost:27020', mongocryptd_bypass_spawn=False, mongocryptd_spawn_path='mongocryptd', mongocryptd_spawn_args=None)

Initialise the secured client:

In [ ]:
secClient = pymongo.MongoClient("127.0.0.1",auto_encryption_opts = fleOptions)

### Insert Data

In [ ]:
employee = {
    'name': 'John Doe',
    'address': {'street': '1234 Main Street',
    'city': 'MongoDBVille',
    'zip': 99999},
    'phone': '949-555-1212',
    'ssn': '123-45-6789',
    'position': {'compensation': 1234}
}

In [ ]:
db = secClient.hr
db.employees.insert_one(employee)

### Read Data

In [ ]:
db = secClient.hr
db.employees.find_one()

### Use Compass or the following to show the encrypted values

In [ ]:
unsecClient = pymongo.MongoClient("127.0.0.1")
db = unsecClient.hr
db.employees.find_one()